<a href="https://colab.research.google.com/github/ryan-saloma/bike-market-summit/blob/main/Bike_Market_Summit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
%pip install -U googlemaps

import googlemaps as gm
import requests

gmaps = gm.Client(key='XXXXXXXXXXXXXXXXX')

# First get location of Summit, NJ
def get_coordinates_from_address(address, api_key):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address': address,
        'key': api_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json().get('results')
        if results:
            location = results[0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print("No results found for the given city name.")
            return None
    else:
        print(f"Error: {response.status_code}")
        return None

summit_trek_coords = \
get_coordinates_from_address('348 Springfield Ave, Summit, NJ 07901', gmaps.key)

# Search for places nearby
nearby_bike_stores = gmaps.places_nearby(location=summit_trek_coords, \
                                         keyword= 'bike store',radius=20000, \
                                         type='bicycle_store')
# Get results as a list of dicts
results = nearby_bike_stores['results']

# Perform list comprehension to extract keys of interest
keys_of_interest = ['name', 'rating', 'user_ratings_total', 'geometry']
stores = [{k: d[k] for k in keys_of_interest} for d in results]

# Sort list in place by rating
stores.sort(key=lambda x: x['rating'], reverse=True)
stores

# NOTE: trek store is not included in "stores"



[{'name': 'Maplewood Bike Shed',
  'rating': 4.9,
  'user_ratings_total': 137,
  'geometry': {'location': {'lat': 40.7218398, 'lng': -74.2601269},
   'viewport': {'northeast': {'lat': 40.72327467989272,
     'lng': -74.25880317010729},
    'southwest': {'lat': 40.72057502010728, 'lng': -74.26150282989273}}}},
 {'name': "Marty's Reliable Cycle of Morristown",
  'rating': 4.9,
  'user_ratings_total': 194,
  'geometry': {'location': {'lat': 40.8036254, 'lng': -74.48219689999999},
   'viewport': {'northeast': {'lat': 40.80498727989271,
     'lng': -74.48098577010727},
    'southwest': {'lat': 40.80228762010727, 'lng': -74.48368542989272}}}},
 {'name': 'Montclair Bikery',
  'rating': 4.9,
  'user_ratings_total': 210,
  'geometry': {'location': {'lat': 40.823204, 'lng': -74.2196403},
   'viewport': {'northeast': {'lat': 40.82468317989272,
     'lng': -74.21817527010728},
    'southwest': {'lat': 40.82198352010727, 'lng': -74.22087492989273}}}},
 {'name': 'Cycle Craft Parsippany',
  'rating':